In [1]:
### changes for this commit
### facebook
### using replica v3 pricing db

In [2]:
import pandas as pd
import pencilbox as pb
from requests import get
import requests

In [3]:
con_rs = pb.get_connection("[Warehouse] Redshift")
con_rep = pb.get_connection("[Replica] Pricing V3")

In [4]:
sql = """

WITH MERCHANT AS
  (SELECT DISTINCT M.id,
                   m.name,
                   m.gr_id,
                   m.enabled_flag,
                   L.NAME AS city
   FROM lake_grofers_db.view_gr_merchant M
   INNER JOIN lake_grofers_db.gr_merchant_additional_info MAI ON MAI.MERCHANT_ID = M.ID
   AND (MAI.MERCHANT_STORE_TYPE = 'grofers')
   INNER JOIN lake_grofers_db.view_gr_locality L2 ON M.LOCALITY_ID = L2.ID
   AND (M.NAME NOT ILIKE '%%GROCERY%%MART%%'
        AND M.NAME NOT ILIKE '%%FRESHBURY%%'
        AND M.NAME NOT ILIKE '%%test%%'
        AND M.NAME NOT ILIKE '%%donation%%')
   INNER JOIN lake_grofers_db.view_gr_locality L1 ON L2.PARENT_LOCALITY_ID = L1.ID
   INNER JOIN lake_grofers_db.view_gr_locality L ON L1.PARENT_LOCALITY_ID = L.ID
   AND L.ID <> 2051 ),
   
   -- fetching mid name, city name and mid details
   
     mapping AS
  ( SELECT DISTINCT vmp.virtual_merchant_id::int AS frontend_id,
                    m1.gr_id AS frontend_gr_id,
                    m1.name AS frontend_name,
                    m1.city AS frontend_city,
                    vmp.real_merchant_id::int AS backend_id,
                    m.gr_id AS backend_gr_id,
                    m.name AS backend_name,
                    m.city AS backend_city,
                    co.outlet_id AS retail_outlet_id,
                    c.name AS retail_outlet_name,
                    x.name AS retail_outlet_city
   FROM lake_grofers_db.gr_virtual_to_real_merchant_mapping vmp
   INNER JOIN merchant m ON m.id=vmp.real_merchant_id
   INNER JOIN merchant m1 ON m1.id=vmp.virtual_merchant_id
   INNER JOIN lake_retail.console_outlet_cms_store co ON co.cms_store=vmp.real_merchant_id
   AND co.active=1
   INNER JOIN lake_retail.console_outlet c ON c.id=co.outlet_id
   LEFT JOIN lake_retail.console_location x ON x.id=c.tax_location_id
   AND m1.enabled_flag=TRUE
   AND vmp.enabled_flag=TRUE
   WHERE x.name IS NOT NULL
     AND x.name <>''
   ORDER BY m1.city ),
   
   -- fetching latest product images
   
     images AS (
     SELECT *,
          row_number() OVER (PARTITION BY product_id
                             ORDER BY update_ts DESC) AS rrank
   FROM lake_grofers_db.gr_product_sliding_image
   WHERE sort_order = 1 ),
   
   -- creating a base of all customers in grofers db
   
   base as (select dim_customer_key as customer_id, cart_checkout_ts_ist,merchant_id, merchant_name,dim_customer_address_key 
   from dwh.fact_sales_order_details fs join dwh.dim_merchant m on 
fs.dim_frontend_merchant_key = m.merchant_key where order_create_ts_ist between current_date-31 and current_Date-1),

-- listing all mids with min 5 orders in the past 2 days

mid_min_condition as (
select
frontend_merchant_id merchant_id,
count(distinct order_id) order_cnt from
dwh.fact_sales_order_item_details
--where order_create_ts_ist >= current_date-1 and order_current_status <> 'CANCELLED'
group by 1
--having order_cnt >= 5
),


 -- matching the most recent order with each mid

merc as (
select  merchant_name, a.merchant_id ,max(cart_checkout_ts_ist) as dt from 
base a
inner join
mid_min_condition b on a.merchant_id = b.merchant_id
group by 1,2),


-- lat long for merchants

merch_lat_long as (select
frontend_merchant_name frontend_name,
backend_merchant_name backend_name,
frontend_merchant_city_name frontend_city,
backend_merchant_city_name backend_city,
b.external_id merchant_id,
a.latitude,
a.longitude,
a.city
from
lake_logistics.logistics_node_address a
inner join
lake_logistics.logistics_node b on a.id = b.node_address_id
inner join 
dwh.dim_merchant_outlet_facility_mapping c on b.external_id = c.frontend_merchant_id
group by 1,2,3,4,5,6,7,8
),

-- matching each mid with users that had the most recent cart checkout and supplying his/ her lat long

SS as (
select 
m.*, b.customer_id,b.dim_customer_address_key, o.latitude location_lat, o.longitude location_lon, o.city
from 
merc m 
inner join 
base b on m.merchant_id = b.merchant_id and m.dt = b.cart_checkout_ts_ist
inner join 
merch_lat_long o on m.merchant_id = o.merchant_id
group by 1,2,3,4,5,6,7,8),

manufacture_info as (
select
a.product_id,
case when a.attribute_id = 632 then a.value else '' end manufacturer_info,
case when b.attribute_id = 641 then ltrim(rtrim(b.value)) else '' end origin_country,
case when d.attribute_id = 634 then ltrim(rtrim(d.value)) else '' end importer_name,
case when d.attribute_id = 634 then ltrim(rtrim(d.value)) else '' end importer_address,
c.Country_code
from 
lake_grofers_db.gr_product_attribute_mapping a
inner join 
lake_grofers_db.gr_product_attribute_mapping b on a.product_id = b.product_id
inner join
metrics.iso_codes c on ltrim(rtrim(b.value)) ilike c.country_name
inner join 
lake_grofers_db.gr_product_attribute_mapping d on a.product_id = d.product_id
where a.attribute_id in (632) and b.attribute_id in (641) and d.attribute_id in (634)
),

-- top selling items

top_selling as (
select product_id from
(select product_id, sum(product_quantity) sold from dwh.fact_sales_order_item_details where 
order_create_ts_ist >current_date-7 and order_current_status <> 'CANCELLED'
group by 1
order by 2 desc)
limit 2600
),


-- for each city, getting the highest price of the product and sharing the lat long of the latest order with that mid

final_agg as (
SELECT 
mpm.product_id as id,
replace(p.name, '%%',' percent')|| ' - '||p.unit as title,
p.name as description,
'http://grofers.s3.amazonaws.com/'||psi.image_path as image_link,
'android-app://com.grofers.customerapp/grofers/product?expr=%%22p'||p.id||'%%22&restricted=false' as mobile_android_app_link,
case when mpm.inventory_limit = 0 then 'out of stock'
when mpm.inventory_limit > 0 then 'in stock' end as availability,
mll.frontend_name AS frontend_name,
mll.merchant_id frontend_id,
case when top_selling.product_id is not null then 'TS' else 'NTS' end  as custom_label_4,
p.brand as brand,
cat.name as google_product_category,
'new' as condition,
'no' as adult,
'no' as identifier_exists,
mll.frontend_city as custom_label_3,
'https://blinkit.com/prn/'||lower(replace(replace(p.name,' ','-'),'%%','percent'))||'/prid/'||p.id||'?lat='||location_lat||'&lon='||location_lon as link,
mpm.price as SP,
mpm.mrp,
mpm.product_id as item_group,
'' as custom_label_1,
'home'|| ' > '||cat.name|| ' > '||cat1.name|| ' > '||cat2.name as product_type,
lower(mll.frontend_city) as city,
location_lat as lat,
location_lon as lon,
'grofers://pdp?productId='||mpm.product_id as ios_url,
'grofers://pdp?productId='||mpm.product_id as android_url,
l0_category,
l1_category,
l2_category,
product_type ptype,
ss.merchant_id,
ss.location_lat,
ss.location_lon,
ss.city city_ss,
case when pam.Country_code is null then 'IN' else pam.Country_code end as origin_country,
case when pam.manufacturer_info  is null then 'HOT' else pam.manufacturer_info  end as manufacturer_info,
case when pam.Country_code is null then null when pam.Country_code = 'IN' then null else pam.importer_name end import_name,
case when pam.Country_code is null then null when pam.Country_code = 'IN' then null else pam.importer_address end importer_address,
row_number() over(partition by mll.frontend_city,mpm.product_id order by mpm.price desc ) as S_no
FROM 
lake_grofers_db.gr_merchant_product_mapping mpm
--left join
--top_selling on mpm.product_id = top_selling.product_id
INNER JOIN 
merch_lat_long mll on mll.merchant_id = mpm.merchant_id 
-and mpm.enabled_flag=true and mpm.inventory_limit>0
INNER JOIN 
lake_grofers_db.gr_product p ON mpm.product_id = p.id--now mapping the category
inner join 
images psi on mpm.product_id = psi.product_id and rrank = 1
INNER JOIN 
lake_grofers_db.gr_product_category_mapping PCM ON P.ID = PCM.PRODUCT_ID
inner join 
ss on mll.merchant_id = ss.merchant_id
INNER JOIN 
lake_grofers_db.gr_category CAT2 ON PCM.CATEGORY_ID = CAT2.ID AND PCM.IS_PRIMARY=TRUE
INNER JOIN 
lake_grofers_db.gr_category CAT1 ON CAT2.PARENT_CATEGORY_ID = CAT1.ID
INNER JOIN 
lake_grofers_db.gr_category CAT ON CAT1.PARENT_CATEGORY_ID = CAT.ID and CAT.id not in (343,909)
inner join 
lake_rpc.item_product_mapping ipm on ipm.product_id=mpm.product_id 
inner join
dwh.dim_product dp on dp.product_id = mpm.product_id 
left join
manufacture_info pam on dp.product_id = pam.product_id
WHERE p.enabled_flag=TRUE
and dp.is_current = True
and psi.sort_order=1
and mpm.price>0
and mpm.mrp is not null
and mll.backend_name not ilike 'Super Store - B2B NCR Warehouse'
and mll.backend_city not ilike 'Haridwar'
and mll.Frontend_City not in ('%%Not in service area%%','durgapur')
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,mpm.update_ts)
   
Select

 item_group,
 frontend_id,
 id,
 title,
 description,
 availability,
 condition,
 mrp||' INR' as price,
 SP||' INR' as sale_price,
 link,
 image_link,
 brand,
 product_type,
 ios_url,
 '960335206' as ios_app_store_id,
 'grofers' as ios_app_name,
 android_url,
 'com.grofers.customerapp' as android_package,
 'grofers' as android_app_name,
 l0_category as custom_label_0,
 l1_category as custom_label_1,
 l2_category as custom_label_2,
 city as custom_label_3,
 custom_label_4,
 adult,
 identifier_exists,
 origin_country,
 manufacturer_info,
 import_name importer_name,
 importer_address
FROM final_agg
where S_no = 1
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30

"""

In [5]:
df = pd.read_sql(sql=sql, con=con_rs)
df.head()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "top_selling" does not exist

[SQL: /* User:prakash.raj@grofers.com QueryHash:25181ee9f76230796666df7cf5b5d43b TaskID:adhoc Source:Pencilbox|JHub */
 

WITH MERCHANT AS
  (SELECT DISTINCT M.id,
                   m.name,
                   m.gr_id,
                   m.enabled_flag,
                   L.NAME AS city
   FROM lake_grofers_db.view_gr_merchant M
   INNER JOIN lake_grofers_db.gr_merchant_additional_info MAI ON MAI.MERCHANT_ID = M.ID
   AND (MAI.MERCHANT_STORE_TYPE = 'grofers')
   INNER JOIN lake_grofers_db.view_gr_locality L2 ON M.LOCALITY_ID = L2.ID
   AND (M.NAME NOT ILIKE '%%GROCERY%%MART%%'
        AND M.NAME NOT ILIKE '%%FRESHBURY%%'
        AND M.NAME NOT ILIKE '%%test%%'
        AND M.NAME NOT ILIKE '%%donation%%')
   INNER JOIN lake_grofers_db.view_gr_locality L1 ON L2.PARENT_LOCALITY_ID = L1.ID
   INNER JOIN lake_grofers_db.view_gr_locality L ON L1.PARENT_LOCALITY_ID = L.ID
   AND L.ID <> 2051 ),
   
   --

In [6]:
df.shape

(85375, 30)

In [11]:
### tobacco products

In [7]:
sql_tob = """

select product_id from
((select
product_id
from dwh.dim_product where is_current = true 
and lower(product_type) in ('paan','Cigarette','Hookah Flavor','Gulkand') )
union all
(select* from metrics.nicotine_products) ) group by 1

"""

df_tob = pd.read_sql(sql=sql_tob, con=con_rs)
df_tob.head()

,product_id
0,172948
1,480096
2,309973
3,480514
4,474055


In [8]:
df_tob_ll = df_tob.product_id.to_list()

In [9]:
df = df[(df.item_group.isin(df_tob_ll) == False)]
df.shape

(84978, 30)

In [10]:
sql = """SELECT cms_product_id item_group, cast(frontend_id as varchar) frontend_id, mrp actual_mrp, price actual_price FROM pricing_domain_pricerecommendation"""

In [11]:
df_prices = pd.read_sql(sql=sql, con=con_rep)
df_prices.head()

,item_group,frontend_id,actual_mrp,actual_price
0,477948,30463,330.0,101.0
1,366098,30981,40.0,32.0
2,350,31294,105.0,90.0
3,127442,29859,109.0,104.0
4,90,29703,85.0,85.0


In [12]:
merged_data = df.merge(df_prices, how="inner", on=["item_group", "frontend_id"])
merged_data.head()

,item_group,frontend_id,id,title,description,availability,condition,price,sale_price,link,...,custom_label_3,custom_label_4,adult,identifier_exists,origin_country,manufacturer_info,importer_name,importer_address,actual_mrp,actual_price
0,97237,29861,Ahmedabad_97237,Similac Advance Stage 2 Infant Formula - 400 g,Similac Advance Stage 2 Infant Formula,in stock,new,560 INR,560 INR,https://blinkit.com/prn/similac-advance-stage-...,...,ahmedabad,NTS,no,no,IN,HOT,None,None,560.0,560.0
1,406285,29861,Ahmedabad_406285,MamyPoko Pants Extra Absorb Diaper (XL) - Pack...,MamyPoko Pants Extra Absorb Diaper (XL) - Pack...,in stock,new,699 INR,614 INR,https://blinkit.com/prn/mamypoko-pants-extra-a...,...,ahmedabad,NTS,no,no,IN,HOT,None,None,699.0,595.0
2,437237,30560,Ahmedabad_437237,Kellogg's Breakfast Cereals Chocos - 385 g,Kellogg's Breakfast Cereals Chocos,in stock,new,170 INR,170 INR,https://blinkit.com/prn/kellogg's-breakfast-ce...,...,ahmedabad,NTS,no,no,IN,"Kelloggs India Pvt. Ltd.- Plot no. L2 & L3, Ta...",None,None,180.0,180.0
3,440107,30591,Ahmedabad_440107,GHH Bakeware Cake Aluminum Mould Set Heart Sha...,GHH Bakeware Cake Aluminum Mould Set Heart Sha...,in stock,new,649 INR,312 INR,https://blinkit.com/prn/ghh-bakeware-cake-alum...,...,ahmedabad,NTS,no,no,IN,"Swadeshi Street\n Rz-22A, Ground Floor, Street...",None,None,649.0,339.0
4,443386,30612,Ahmedabad_443386,Slurrp Farm Millet Supergrains + Spinach Dosa ...,Slurrp Farm Millet Supergrains + Spinach Dosa ...,in stock,new,89 INR,87 INR,https://blinkit.com/prn/slurrp-farm-millet-sup...,...,ahmedabad,NTS,no,no,IN,"Christy Foods, 1/374B, Chettikadu, Andipalayam...",None,None,89.0,81.0


In [13]:
merged_data.shape

(81157, 32)

In [14]:
merged_data["price"] = merged_data["actual_mrp"].astype(int).astype(str) + " INR"
merged_data["sale_price"] = merged_data["actual_price"].astype(int).astype(str) + " INR"
df = merged_data.copy()
df.head()

,item_group,frontend_id,id,title,description,availability,condition,price,sale_price,link,...,custom_label_3,custom_label_4,adult,identifier_exists,origin_country,manufacturer_info,importer_name,importer_address,actual_mrp,actual_price
0,97237,29861,Ahmedabad_97237,Similac Advance Stage 2 Infant Formula - 400 g,Similac Advance Stage 2 Infant Formula,in stock,new,560 INR,560 INR,https://blinkit.com/prn/similac-advance-stage-...,...,ahmedabad,NTS,no,no,IN,HOT,None,None,560.0,560.0
1,406285,29861,Ahmedabad_406285,MamyPoko Pants Extra Absorb Diaper (XL) - Pack...,MamyPoko Pants Extra Absorb Diaper (XL) - Pack...,in stock,new,699 INR,595 INR,https://blinkit.com/prn/mamypoko-pants-extra-a...,...,ahmedabad,NTS,no,no,IN,HOT,None,None,699.0,595.0
2,437237,30560,Ahmedabad_437237,Kellogg's Breakfast Cereals Chocos - 385 g,Kellogg's Breakfast Cereals Chocos,in stock,new,180 INR,180 INR,https://blinkit.com/prn/kellogg's-breakfast-ce...,...,ahmedabad,NTS,no,no,IN,"Kelloggs India Pvt. Ltd.- Plot no. L2 & L3, Ta...",None,None,180.0,180.0
3,440107,30591,Ahmedabad_440107,GHH Bakeware Cake Aluminum Mould Set Heart Sha...,GHH Bakeware Cake Aluminum Mould Set Heart Sha...,in stock,new,649 INR,339 INR,https://blinkit.com/prn/ghh-bakeware-cake-alum...,...,ahmedabad,NTS,no,no,IN,"Swadeshi Street\n Rz-22A, Ground Floor, Street...",None,None,649.0,339.0
4,443386,30612,Ahmedabad_443386,Slurrp Farm Millet Supergrains + Spinach Dosa ...,Slurrp Farm Millet Supergrains + Spinach Dosa ...,in stock,new,89 INR,81 INR,https://blinkit.com/prn/slurrp-farm-millet-sup...,...,ahmedabad,NTS,no,no,IN,"Christy Foods, 1/374B, Chettikadu, Andipalayam...",None,None,89.0,81.0


In [15]:
del df["frontend_id"]
del df["item_group"]
del df["actual_mrp"]
del df["actual_price"]
df.shape

(81157, 28)

In [16]:
### checking if all df is non empty
assert not (df.empty or df[df.availability == "in stock"].empty)

In [17]:
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81157 entries, 0 to 81156
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 81157 non-null  object
 1   title              81157 non-null  object
 2   description        81157 non-null  object
 3   availability       81157 non-null  object
 4   condition          81157 non-null  object
 5   price              81157 non-null  object
 6   sale_price         81157 non-null  object
 7   link               81157 non-null  object
 8   image_link         81157 non-null  object
 9   brand              80713 non-null  object
 10  product_type       81157 non-null  object
 11  ios_url            81157 non-null  object
 12  ios_app_store_id   81157 non-null  object
 13  ios_app_name       81157 non-null  object
 14  android_url        81157 non-null  object
 15  android_package    81157 non-null  object
 16  android_app_name   81157 non-null  objec

In [18]:
print(df.shape)

(81157, 28)


In [24]:
### adding dag run flag condition

In [25]:
sheet_id = "1-uNp4c9uuGrWibaG2MHThepvx-o4Jctke3DpeS5Zsxg"
sheet_name = "flags"
flags_df = pb.from_sheets(sheet_id, sheet_name)
var = flags_df[flags_df.Name == "facebook_s3"]

In [26]:
var.Flag.to_list()[0] == "1"

True

In [27]:
df.head()

,id,title,description,availability,condition,price,sale_price,link,image_link,brand,...,custom_label_1,custom_label_2,custom_label_3,custom_label_4,adult,identifier_exists,origin_country,manufacturer_info,importer_name,importer_address
0,Ahmedabad_313352,Go Pizza Cheese Block - 400 g,Go Pizza Cheese Block,in stock,new,249 INR,249 INR,https://blinkit.com/prn/go-pizza-cheese-block/...,http://grofers.s3.amazonaws.com/app/images/pro...,Go,...,Cheese,Cheese,ahmedabad,NTS,no,no,IN,HOT,None,None
1,Ahmedabad_446662,Dabur Vatika Anti-Dandruff Shampoo - 640 ml,Dabur Vatika Anti-Dandruff Shampoo,in stock,new,380 INR,255 INR,https://blinkit.com/prn/dabur-vatika-anti-dand...,http://grofers.s3.amazonaws.com/app/images/pro...,Dabur,...,Shampoo & Conditioner,Shampoo & Conditioner,ahmedabad,NTS,no,no,IN,"Dabur India Ltd , (Unit -2 ) , Integrated Indu...",None,None
2,Bengaluru_11350,Johnson's Baby Lotion - 100 ml,Johnson's Baby Lotion,in stock,new,95 INR,19 INR,https://blinkit.com/prn/johnson's-baby-lotion/...,http://grofers.s3.amazonaws.com/app/images/pro...,Johnson's,...,Skin Care,Skin Care,bengaluru,NTS,no,no,IN,HOT,None,None
3,Bengaluru_24380,Olay Age Protect Face Cream - 40 g,Olay Age Protect Face Cream,in stock,new,349 INR,301 INR,https://blinkit.com/prn/olay-age-protect-face-...,http://grofers.s3.amazonaws.com/app/images/pro...,Olay,...,Facial Care,Face Cream & Lotion,bengaluru,NTS,no,no,IN,"Procter & Gamble. Katha, Himachal Pradesh 173205",None,None
4,Bengaluru_35484,Keya Creamy Mushroom Soup - 44 g,Keya Creamy Mushroom Soup,in stock,new,60 INR,40 INR,https://blinkit.com/prn/keya-creamy-mushroom-s...,http://grofers.s3.amazonaws.com/app/images/pro...,Keya,...,"Noodles, Pasta & Soup",Soup,bengaluru,NTS,no,no,IN,HOT,None,None


In [36]:
list_ll = df.id.to_list()
pid_set = set()
for i in range(len(list_ll)):
    pid_set.add(list_ll[i].split('_')[1])

len(pid_set)

2445

In [38]:
for city, city_df in df.groupby("custom_label_3"):
    filename = city.lower() + "_top_selling.csv"
    filepath = f"/tmp/{filename}"
    city_df.to_csv(filepath, index=False)
    print(
        "Dumping data for {city} to {filename} with size : {size}".format(
            city=city, filename=filename, size=city_df.shape
        )
    )
    if var.Flag.to_list()[0] == "1":
        print("Running now")
        pb.to_s3(filepath, "grofers-facebook-product-feed", filename)
    else:
        print("No runtime due to flag")

Dumping data for ahmedabad to ahmedabad_top_selling.csv with size : (1374, 28)
Running now
Dumping data for bengaluru to bengaluru_top_selling.csv with size : (1808, 28)
Running now
Dumping data for chandigarh to chandigarh_top_selling.csv with size : (1127, 28)
Running now
Dumping data for chennai to chennai_top_selling.csv with size : (1150, 28)
Running now
Dumping data for delhi to delhi_top_selling.csv with size : (2078, 28)
Running now
Dumping data for faridabad to faridabad_top_selling.csv with size : (1949, 28)
Running now
Dumping data for hr-ncr to hr-ncr_top_selling.csv with size : (2050, 28)
Running now
Dumping data for hyderabad to hyderabad_top_selling.csv with size : (1713, 28)
Running now
Dumping data for jaipur to jaipur_top_selling.csv with size : (1457, 28)
Running now
Dumping data for jalandhar to jalandhar_top_selling.csv with size : (1275, 28)
Running now
Dumping data for kanpur to kanpur_top_selling.csv with size : (1471, 28)
Running now
Dumping data for kolkata to